In [21]:
import cv2 as cv
import numpy as np

inputMode = True


cam = cv.VideoCapture(0)
ret, frame = cam.read()

x, y = 0, 0
height, width = 25, 25
track_window = (x,y,width,height)
roi = frame[y:y + height, x:x + width]

# callback del mouse
def selectROI(event, x, y, flags, param):
    global frame, roi, inputMode
    if inputMode and event==cv.EVENT_LBUTTONDOWN and len(roi) < 4:
        roi.append((x, y))
        cv.circle(frame, (x, y), 4, (0, 255, 0), 2)
        cv.imshow('Cam', frame)

# mask
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

# criterio de terminación
term_crit = ( cv.TERM_CRITERIA_COUNT | cv.TERM_CRITERIA_EPS, 10, 1 )

cv.namedWindow('Cam')  
cv.setMouseCallback('Cam',selectROI)  

while True:
   
    ret, frame = cam.read()

    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        x,y,w,h = track_window
        img = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv.imshow('Cam', img)

        k = cv.waitKey(60) & 0xff
        if k == 27:
            break
        else:
            cv.imwrite(chr(k)+".jpg",img)
    else:
        break

cv.destroyAllWindows()
cam.release()